# End to End testing with python-requests
- http://docs.python-requests.org/en/master
- http://docs.python-requests.org/en/master/user/quickstart/#make-a-request

## BEFORE YOU RUN THESE TESTS: 
- Delete app.db
- Start the server, which will create a fresh database, with a default Admin, Usermanager and User

### Set some global variables & import modules

In [ ]:
SERVER = 'http://localhost:5000'

import requests
import json
from base64 import b64encode
from datetime import datetime, date, time

from io import BytesIO
from PIL import Image

import tempfile

from tests_utils import pprint_sequence
from e2e_tests_utils import pprint_request, pprint_response

### Define some helper functions & global variables

In [ ]:
def get_api_headers(email, password):
    return {
        'Authorization': 'Basic ' + b64encode(
            (email + ':' + password).encode('utf-8')).decode('utf-8'),
        'Content-Type': 'application/vnd.api+json',
        'Accept': 'application/vnd.api+json'
    }

def get_api_headers_multiform(email, password):
    return {
        'Authorization': 'Basic ' + b64encode(
            (email + ':' + password).encode('utf-8')).decode('utf-8'),
        'Content-Type': 'multipart/form-data',
        'Accept': 'application/vnd.api+json'
    }

def pprint_request_and_response(r,
                                print_request_body=True,
                                print_response_text=True,
                                print_response_content=True):
    print('=================================')
    print('===THE REQUEST WE SENT===')
    print('=================================')
    pprint_request(r.request, print_body=print_request_body)
    print('=================================')
    print('===THE RESPONSE WE RECEIVED===')
    print('=================================')
    pprint_response(r, 
                    print_text=print_response_text, 
                    print_content=print_response_content)

# to get access to instance/config.py
import e2e_my_file_location
from instance.config import ADMIN_EMAIL, ADMIN_PW,\
                            USERMANAGER_EMAIL, USERMANAGER_PW,\
                            USER_EMAIL, USER_PW,\
        NUTRINIONIX_APP_ID, NUTRINIONIX_APP_KEY
        

### TEST: Register myself as a user

In [ ]:
url = SERVER+'/api/v1/users/'
headers = {
    'Content-Type': 'application/vnd.api+json',
    'Accept': 'application/vnd.api+json'
    }
payload = {
    "data": {
        "type": "user",
        "attributes": {
            "email": 'arjaan.buijk@gmail.com',
            "password": "a_real_password",
            "first_name": "Arjaan",
            "last_name": "Buijk",
        }
    }
}

r = requests.post(url, headers=headers, json=payload)

In [ ]:
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

#### Store link to myself, for usage below.

In [ ]:
json_response = json.loads(r.text)
url_to_self = json_response['links']['self']
print('url_to_self = {}'.format(url_to_self))

### TEST: Without activating account, getting a token must be denied

In [ ]:
url = SERVER+'/api/v1/token'
headers = get_api_headers('arjaan.buijk@gmail.com',
                          'a_real_password')

In [ ]:
r = requests.post(url, headers=headers)

In [ ]:
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

### NOTE: Activate Account before running next test:
- check email to activate the account
- after activating the account, logout from the website

## TEST: get token now that account is activated

In [ ]:
url = SERVER+'/api/v1/token'
headers = get_api_headers('arjaan.buijk@gmail.com',
                          'a_real_password')
r = requests.post(url, headers=headers)
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

### Store the token for usage below

In [ ]:
json_response = json.loads(r.text)
token = json_response['token']
print('token = {}'.format(token))

---
## ENTERING ITEMS

In [ ]:
url = SERVER+'/api/v1/items/'
headers = get_api_headers(token,'')
data = {
    "data": {
        "type": "item",
        "attributes": {
            "date": '{}'.format(date(2018,1,5)),
            "time": '{}'.format(time(18,5,15)),
            "description": "TODO!!!!"
        }
    }
}
r = requests.post(url, headers=headers, json=data)
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

### Verify only Admin, Usermanager and myself can access my data, not another regular user

In [ ]:
url = SERVER+url_to_self

headers = get_api_headers(ADMIN_EMAIL,ADMIN_PW)
r = requests.get(url, headers=headers)
print('Retrieve my data as Admin. Status code = {}'.format(r.status_code))

headers = get_api_headers(USERMANAGER_EMAIL,USERMANAGER_PW)
r = requests.get(url, headers=headers)
print('Retrieve my data as Usermanager. Status code = {}'.format(r.status_code))

headers = get_api_headers(token,'')
r = requests.get(url, headers=headers)
print('Retrieve my data as myself. Status code = {}'.format(r.status_code))

headers = get_api_headers(USER_EMAIL,USER_PW)
r = requests.get(url, headers=headers)
print('Retrieve my data as other user. Status code = {}'.format(r.status_code))


### Upload a profile picture, using multipart/form-data type request

In [ ]:
url = SERVER+'/api/v1/profile_pic'
r = requests.post(url, auth=(token,''), files={'profile_pic': open('test_profile_pic.gif', 'rb')})

In [ ]:
pprint_request_and_response(r,
                            print_request_body=False,
                            print_response_text=True,
                            print_response_content=False)

---
### Retrieve the profile picture

In [ ]:
url = SERVER+'/api/v1/profile_pic'
r = requests.get(url, auth=(token,''))

In [ ]:
pprint_request_and_response(r,
                            print_request_body=False,
                            print_response_text=False,
                            print_response_content=False)

#### Store the profile picture in a Pillow Image, and then
- Display it
- Save it to disk

In [ ]:
img = Image.open(BytesIO(r.content))

In [ ]:
img.show()

In [ ]:
fh, filename = tempfile.mkstemp(suffix='.gif', prefix='profile_pic_')
img.save(filename)
print('Saved profile picture to {}'.format(filename))

---
### Lock my account by entering wrong password 3 times

In [ ]:
url = SERVER+'/api/v1/token'
headers = get_api_headers('arjaan.buijk@gmail.com',
                          'a_wrong_password')
for i in range(4):
    r = requests.post(url, headers=headers)
    print('Wrong attempt {} gives status_code={}'.format(i,r.status_code))

In [ ]:
pprint_request_and_response(r,
                            print_request_body=False,
                            print_response_text=True,
                            print_response_content=False)

---
### Admin can unlock the account

In [ ]:
url = SERVER+url_to_self+'/unblock'
headers = get_api_headers(ADMIN_EMAIL,ADMIN_PW)

In [ ]:
r = requests.post(url, headers=headers)

In [ ]:
pprint_request_and_response(r,
                            print_request_body=False,
                            print_response_text=True,
                            print_response_content=False)

#### Verify it is unlocked, by retrieving a new token

In [ ]:
url = SERVER+'/api/v1/token'
headers = get_api_headers('arjaan.buijk@gmail.com',
                          'a_real_password')
r = requests.post(url, headers=headers)
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)
json_response = json.loads(r.text)
token = json_response['token']
print('token = {}'.format(token))

---
### Delete my account

In [ ]:
url = SERVER+url_to_self
headers = get_api_headers('arjaan.buijk@gmail.com',
                          'a_real_password')
r = requests.delete(url, headers=headers)
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

---
### As admin, invite me to join

In [ ]:
url = SERVER+'/api/v1/invite/arjaan.buijk@gmail.com'
headers = get_api_headers(ADMIN_EMAIL,ADMIN_PW)

In [ ]:
r = requests.post(url, headers=headers)

In [ ]:
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

### NOTE: Finish Account Registration before running next test:
- check email to finish the account registration
- after activating the account, logout from the website

### Verify that account is active, by getting a new token

In [ ]:
url = SERVER+'/api/v1/token'
headers = get_api_headers('arjaan.buijk@gmail.com',
                          'a_real_password')
r = requests.post(url, headers=headers)
json_response = json.loads(r.text)
token = json_response['token']
print('token = {}'.format(token))

---
## Filtering

### Get user's info via a filter object in query string

In [ ]:
qs=[{"name":"email","op":"eq","val":"{}".format("arjaan.buijk@gmail.com")}]
url = SERVER+'/api/v1/users?filter='+json.dumps(qs)
headers = get_api_headers(ADMIN_EMAIL,ADMIN_PW)
r = requests.get(url, headers=headers)
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

### 'or' filter on users

In [ ]:
# 'or' filter on users
email1 = "arjaan.buijk@gmail.com"
email2 = USER_EMAIL
qs=[{"or":[{"name":"email","op":"eq","val":"{}".format(email1)},
           {"name":"email","op":"eq","val":"{}".format(email2)}
           ]
     }
    ]
url = SERVER+'/api/v1/users?filter='+json.dumps(qs)
headers = get_api_headers(ADMIN_EMAIL,ADMIN_PW)
r = requests.get(url, headers=headers)
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)


### Fancy filter 

In [ ]:
qs=[ { 'and': [	{'or': [{'name': 'date', 'op': 'eq', 'val': '2018-01-05'},
                        {'name': 'date', 'op': 'eq', 'val': '2018-01-10'}
                       ]
                },
                {'or': [{'name': 'calories', 'op': 'gt', 'val': '300.0'},
                        {'name': 'calories', 'op': 'lt', 'val': '1000.0'}
                       ]
                }
              ]
     }
   ]

url = SERVER+'/api/v1/meals?filter='+json.dumps(qs)
headers = get_api_headers(ADMIN_EMAIL,ADMIN_PW)
r = requests.get(url, headers=headers)
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)


---
# SANDBOX

### Testing nutritionix api

In [ ]:
app_id = NUTRINIONIX_APP_ID
app_key = NUTRINIONIX_APP_KEY
description = "Potatoes, a Veggie Patty and a Milk"

url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
headers = {
    'Content-Type': 'application/json',
    'x-app-id': '{}'.format(app_id),
    'x-app-key': '{}'.format(app_key),
    'Accept': 'application/vnd.api+json'
    }
payload = {
    "query": description
}

r = requests.post(url, headers=headers, json=payload)

In [ ]:
pprint_request_and_response(r,
                            print_request_body=True,
                            print_response_text=True,
                            print_response_content=False)

In [ ]:
r_json = json.loads(r.text)
foods = r_json['foods']
total_calories = 0
for food in foods:
    print('Calories in {} is {}'.format(food['food_name'], food['nf_calories']))
    total_calories += food['nf_calories']
print('_____________________________________________________________________ +')
print('Total calories of this meal is {}'.format(total_calories))

### Available methods and attributes for request

In [ ]:
dir(r.request)

### Available methods and attributes for response

In [ ]:
dir(r)

In [ ]:
print (url)

In [ ]:
print(r.content)

In [ ]:
print (r.text)

In [ ]:
open('test_profile_pic.gif', 'rb')


In [ ]:
import os.path
os.path.isfile('test_profile_pic.gif')


In [ ]:
img = Image.open(StringIO(r.text))